<a href="https://colab.research.google.com/github/WangKehanK/CS505/blob/main/Twitter_User_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import json
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Twitter_user_handles_to_predict.csv", encoding = "utf-8")
df_labeled = pd.read_csv("/content/drive/My Drive/Twitter_users_labeled_with_age_and_gender.csv", encoding = "latin-1")

In [4]:
# Get authentication for tweepy from auth.txt
# auth.txt was uploaded before in order to use m3inference
keys = ''
with open(f'/content/drive/MyDrive/auth.txt', 'r') as reader:
  keys=reader.read()
keys = keys.split('\n')
# for i in range(len(keys)):
#   keys[i] = keys[i].split('"')[1];


In [5]:
# Setup tweepy
import tweepy
api_key = keys[0]
api_secret = keys[1]
access_token = keys[2]
access_secret = keys[3]
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [6]:
# Try first user
user = df['Username']
userID = user[0]
print(userID, len(user))
tweets = api.user_timeline(screen_name=userID,count=100,include_rts = False,tweet_mode = 'extended')

JayHolz410 25129


In [ ]:
# Extract full text from tweets
# for testing purpose, print user's 3 most recent tweets

tweet_text = [tweet._json['full_text'] for tweet in tweets]
for info in tweets[:3]:
    print("ID: {}".format(info.id))
    print(info.created_at)
    print(info.full_text)
    print("\n")
print(len(tweet_text))

In [ ]:
# Extract latest 100 tweets using api.user_timeline for each user
import time

all_tweets = []
for sleep_count, userID in enumerate(user):
  try:
    tweets = api.user_timeline(screen_name=userID, 
                              count=100,
                              include_rts = False,
                              tweet_mode = 'extended')
    all_tweets.extend(tweets)
    print('Number of tweets extracted {}'.format(len(all_tweets)))

    if (sleep_count %400 == 0 and not sleep_count == 0):
      time.sleep(15*60)
      print("Taking a sleep break..." + '\n')
      
  except tweepy.TweepError as e:
    # print(e.response.status_code, e.reason)

    # the rate limit error
    if e.response.status_code == 88:
       time.sleep(15*60)
       print("Taking a sleep break..." + '\n')

    # the user not found error
    else: 
      print("screen_name that failed=",  userID)
      pass

print("Extraction Done")

Number of tweets extracted 91
Number of tweets extracted 159
Number of tweets extracted 244
Number of tweets extracted 290
Number of tweets extracted 340
Number of tweets extracted 406
Number of tweets extracted 474
Number of tweets extracted 538
Number of tweets extracted 581
Number of tweets extracted 610
screen_name that failed= TheDailyOoze
Number of tweets extracted 693
Number of tweets extracted 762
screen_name that failed= Kiswan93
Number of tweets extracted 823
screen_name that failed= KylahGuion
Number of tweets extracted 863
Number of tweets extracted 960
screen_name that failed= DaRellCafe
Number of tweets extracted 1054
Number of tweets extracted 1097
Number of tweets extracted 1112
Number of tweets extracted 1176
screen_name that failed= 5hunnidPheebz
Number of tweets extracted 1255
Number of tweets extracted 1329
screen_name that failed= RichDame1
Number of tweets extracted 1411
Number of tweets extracted 1503
Number of tweets extracted 1565
Number of tweets extracted 166

Rate limit reached. Sleeping for: 189


Number of tweets extracted 22478
Number of tweets extracted 22544
Number of tweets extracted 22614
screen_name that failed= JasmineOrozco3
Number of tweets extracted 22697
Number of tweets extracted 22728
Taking a sleep break...

screen_name that failed= tysosaucy23
Number of tweets extracted 22732
screen_name that failed= madinhoeee
screen_name that failed= _princessxans
screen_name that failed= BrownBwoySolo
Number of tweets extracted 22749
Number of tweets extracted 22791
Number of tweets extracted 22838
Number of tweets extracted 22840
Number of tweets extracted 22874
Number of tweets extracted 22924
Number of tweets extracted 22924
Number of tweets extracted 22992
Number of tweets extracted 23042
Number of tweets extracted 23094
screen_name that failed= _Kuzimaki
Number of tweets extracted 23175
Number of tweets extracted 23233
Number of tweets extracted 23286
Number of tweets extracted 23310
Number of tweets extracted 23333
Number of tweets extracted 23333
Number of tweets extrac

Rate limit reached. Sleeping for: 604


screen_name that failed= TheBurntCracker
screen_name that failed= oen_kyle
Number of tweets extracted 43258
Number of tweets extracted 43281
Number of tweets extracted 43307
screen_name that failed= Yg_Kyrie
Number of tweets extracted 43320
Number of tweets extracted 43357
Number of tweets extracted 43366
screen_name that failed= TravPesci
Number of tweets extracted 43366
screen_name that failed= fuckantho
screen_name that failed= meekwannaminaj
Number of tweets extracted 43458
screen_name that failed= 1ofthemOnes
Number of tweets extracted 43535
Number of tweets extracted 43611
Number of tweets extracted 43690
Number of tweets extracted 43762
screen_name that failed= YGURTMAN
screen_name that failed= Thugga2x
screen_name that failed= SimboKlice
Number of tweets extracted 43798
Number of tweets extracted 43844
Number of tweets extracted 43879
Number of tweets extracted 43956
Number of tweets extracted 44041
Number of tweets extracted 44070
Number of tweets extracted 44153
Number of twe

Rate limit reached. Sleeping for: 639


Number of tweets extracted 60852
screen_name that failed= ItsNieme
Number of tweets extracted 60894
Number of tweets extracted 60900
screen_name that failed= reppin4r
screen_name that failed= sleazeballl
Number of tweets extracted 60970
screen_name that failed= Vickitoria_Finn
Number of tweets extracted 61048
Number of tweets extracted 61124
screen_name that failed= bananalilly
Number of tweets extracted 61177
Number of tweets extracted 61238
screen_name that failed= j__danielle
Number of tweets extracted 61238
Number of tweets extracted 61275
Number of tweets extracted 61283
screen_name that failed= Mattheww___
Number of tweets extracted 61286
screen_name that failed= belllllssssssss
screen_name that failed= igigvvul
screen_name that failed= arctic_tic
Number of tweets extracted 61286
Number of tweets extracted 61363
Number of tweets extracted 61366
screen_name that failed= catluver97
Number of tweets extracted 61454
Number of tweets extracted 61467
Number of tweets extracted 61497
Nu

In [ ]:
outtweets = [[tweet.id_str, 
              tweet.created_at, 
              tweet.favorite_count, 
              tweet.retweet_count, 
              tweet.full_text.encode("utf-8").decode("utf-8")] 
             for idx,tweet in enumerate(all_tweets)]

df = DataFrame(outtweets,columns=["id","created_at","favorite_count","retweet_count", "text"])
df.to_csv('%all_tweets.csv' % userID,index=False)
df.head(3)


TODO: Using BERT to extract fixed feature vectors

https://github.com/google-research/bert#using-bert-to-extract-fixed-feature-vectors-like-elmo

In [ ]:
age_no_na = df_labeled[df_labeled['human.labeled.age'].notna()]
age_no_na